In [1]:
# import sys
# sys.path.append("/home/baris/repos/rlvr")

# from scripts.musique import evaluate

In [2]:
from pathlib import Path

import typer
import pandas as pd
import verifiers as vf
from dotenv import load_dotenv
from openai import AsyncOpenAI


assert load_dotenv(), "Failed to load .env file"

In [3]:
import mlflow

# Tell MLflow about the server URI.
mlflow.set_tracking_uri("http://127.0.0.1:5000")
# Enable autologging with all features
mlflow.openai.autolog()
# Create a unique name for your experiment.
mlflow.set_experiment("rlvr-eval")

<Experiment: artifact_location='mlflow-artifacts:/5', creation_time=1756107462035, experiment_id='5', last_update_time=1756107462035, lifecycle_stage='active', name='rlvr-eval', tags={}>

In [4]:
openai_client = AsyncOpenAI()

models_response = await openai_client.models.list()
available_models = [item.id for item in models_response.data if item.object == "model"]
print("Available models:", available_models)

model = available_models[0]
print("Using model:", model)


Available models: ['Qwen/Qwen2.5-3B-Instruct']
Using model: Qwen/Qwen2.5-3B-Instruct


In [5]:
datasets_str = "bdsaglam/musique-mini,answerable,validation[:8]"
noise_rate = 1.0
retriever = "hybrid"
temperature = 0.1
max_new_tokens = 1024

print("🔮 Starting MuSiQue evaluation")
print("=" * 50)
print(f"📝 Model: {model}")
print(f"📊 Dataset: {datasets_str} (noise rate: {noise_rate})")
print(f"🔍 Retriever: {retriever}")
print(f"🌡️ Temperature: {temperature}")
print(f"🎯 Max tokens: {max_new_tokens}")
print("=" * 50)

# Load MuSiQue environment
print("🌍 Loading MuSiQue environment...")

vf_env = vf.load_environment(
    env_id="vf-musique",
    datasets_str=datasets_str,
    noise_rate=noise_rate,
    retriever_name=retriever,
)
print(f"✅ Environment loaded with {len(vf_env.dataset)} examples")

# Use OpenAI-compatible API client (e.g., for vLLM)
print("🤖 Using OpenAI-compatible API client...")

# Run evaluation using the environment
print("🔄 Running evaluation...")
results = vf_env.evaluate(
    openai_client,
    model,
    rollouts_per_example=1,
    sampling_args={"temperature": temperature, "max_tokens": max_new_tokens},
)


🔮 Starting MuSiQue evaluation
📝 Model: Qwen/Qwen2.5-3B-Instruct
📊 Dataset: bdsaglam/musique-mini,answerable,validation[:8] (noise rate: 1.0)
🔍 Retriever: hybrid
🌡️ Temperature: 0.1
🎯 Max tokens: 1024
🌍 Loading MuSiQue environment...


RuntimeError: Failed to load environment 'vf-musique': module 'verifiers' has no attribute 'StatefulToolEnv'

In [ ]:
len(results.prompt), len(results.completion), len(results.answer), len(results.state), len(results.info), len(results.task), len(results.reward)

(8, 8, 8, 8, 8, 8, 8)

In [ ]:
results.metrics.keys()

dict_keys(['exact_match_reward', 'f1_reward', 'retrieval_recall_reward', 'citation_reward', 'format_reward', 'combined_reward'])

In [ ]:
from verifiers.types import GenerateOutputs

for field in GenerateOutputs.__fields__.keys():
    print(field)

    _type = type(getattr(results, field))
    if _type == list:
        print(getattr(results, field)[0])
    else:
        print(type(getattr(results, field)))

prompt
[{'role': 'system', 'content': "Answer the question based on the information provided by tools.\n\nFor each step:\n1. Think through your reasoning inside <think> tags\n2. Use tools to retrieve relevant documents\n3. Continue until you have found the answer through multi-hop reasoning\n4. In the **last** step:\n    - Reflect on your previous steps inside <think> tags\n    - Cite the documents you used inside <cite> tags by their IDs, e.g. `<cite>1, 2, 3</cite>`\n    - Give your final answer inside <answer> tags\nAn example for your final message:\n```\n<think>\n[your thinking here]\n</think> \n<cite>\n[IDs of the documents that back your answer]\n</cite>\n<answer>\n[your final answer in a few words]\n</answer>\n```\n\n- Do not make up tools or arguments that aren't listed.\n- Questions require multi-hop reasoning across multiple documents.\n- Continue searching until you find all relevant information to answer the question."}, {'role': 'user', 'content': 'Who\'s the sibling of th

/tmp/ipykernel_687769/3696243233.py:3: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  for field in GenerateOutputs.__fields__.keys():


In [ ]:
ds = vf_env.make_dataset(results)
ds

Dataset({
    features: ['prompt', 'completion', 'answer', 'task', 'reward', 'info', 'exact_match_reward', 'f1_reward', 'retrieval_recall_reward', 'citation_reward', 'format_reward', 'combined_reward'],
    num_rows: 8
})

In [ ]:
ds.to_json("../outputs/sample.jsonl", orient="records", lines=True)

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

266900

In [ ]:
df = ds.to_pandas()

In [ ]:
metric_cols = [
    'exact_match_reward',
    'f1_reward',
    'retrieval_recall_reward',
    'citation_reward',
    'format_reward',
    'combined_reward',
]
df[metric_cols].describe()

,exact_match_reward,f1_reward,retrieval_recall_reward,citation_reward,format_reward,combined_reward
count,8.0,8.000000,8.0,8.000000,8.000000,8.000000
mean,0.0,0.041667,0.0,0.062500,0.416667,0.032639
std,0.0,0.117851,0.0,0.176777,0.295468,0.040980
min,0.0,0.000000,0.0,0.000000,0.000000,0.000000
25%,0.0,0.000000,0.0,0.000000,0.333333,0.011111
50%,0.0,0.000000,0.0,0.000000,0.333333,0.011111
75%,0.0,0.000000,0.0,0.000000,0.416667,0.040278
max,0.0,0.333333,0.0,0.500000,1.000000,0.122222
